In [6]:
import requests
import time
import hashlib
from neo4j import GraphDatabase
import json

In [7]:
API_PUBLIC_KEY = "929b4b43b3ab59b27417573fcd5868b9"
API_PRIVATE_KEY = "27bc1b826da0c2def3525f69a2668e7834744938"
API_LIMIT = 100
API_OFFSET = 0

In [8]:
current_timestamp = time.time()
hash = hashlib.md5((str(current_timestamp)+API_PRIVATE_KEY+API_PUBLIC_KEY).encode())

In [11]:
response_API = requests.get('https://gateway.marvel.com:443/v1/public/characters?limit=%s&offset=%s&apikey=929b4b43b3ab59b27417573fcd5868b9&hash=%s&ts=%s'%(API_LIMIT,API_OFFSET,hash.hexdigest(),str(current_timestamp)))
with open("test_chiamata.json","w") as out_file:
  out_file.write(response_API.text)


In [9]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j","1234qwer"))

In [66]:

def add_character_node(tx, name, character_id):
  name = name.replace('"'," ")
  query_string = 'CREATE (c:character{name:"%s", character_id: %s})'%(name, character_id)
  print(query_string)
  result = tx.run(query_string)

In [67]:
def add_comic_node_and_link(tx, comic_title, comic_id, character_id):
  create_comic_node_query = 'MERGE (c:comic {title:"%s", comic_id: %s})'%(comic_title, comic_id)
  create_link_query = "MATCH (a:character),(b:comic) WHERE a.character_id = %s AND b.comic_id = %s MERGE (a)-[r:Presente]->(b)"%(character_id, comic_id)
  print(create_comic_node_query)
  # print(create_link_query)
  node_creation_result = tx.run(create_comic_node_query)
  link_creation_result = tx.run(create_link_query)

In [68]:
def add_comic_node(tx, comic_title, comic_id):
  create_comic_node_query = 'MERGE (c:comic {title:"%s", comic_id: %s})'%(comic_title, comic_id)
  node_creation_result = tx.run(create_comic_node_query)


Get all marvel comics characters

In [ ]:
results_ended = False
iteration_number = 0
driver = GraphDatabase.driver(
    "bolt://localhost:7687", auth=("neo4j", "1234qwer"))
API_OFFSET = 0
while not results_ended:
    print("********Starting iteration n. %s \n" % (str(iteration_number)))
    response_API = requests.get('https://gateway.marvel.com:443/v1/public/characters?limit=%s&offset=%s&apikey=929b4b43b3ab59b27417573fcd5868b9&hash=%s&ts=%s' %
                                (API_LIMIT, API_OFFSET, hash.hexdigest(), str(current_timestamp)))
    response_JSON = json.loads(response_API.text)
    try:
        if len(response_JSON['data']['results']) != 0:
            for character in response_JSON['data']['results']:
                with driver.session() as session:
                    try:
                        session.write_transaction(
                            add_character_node, character['name'], character["id"])
                    except Exception:
                        print(Exception)
            API_OFFSET += API_LIMIT
            print("********Ending iteration n. %s \n" %
                  (str(iteration_number)))
            iteration_number += 1
        else:
            results_ended = True
    except Exception:
        print(Exception)


Get all Marvel Comics and link characters

In [72]:
results_ended = False
iteration_number = 0
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j","1234qwer"))
API_OFFSET = 0
API_LIMIT = 100
response_JSON = None

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j","1234qwer"))
while not results_ended and iteration_number < 5:
  response_API = requests.get('https://gateway.marvel.com:443/v1/public/comics?limit=%s&offset=%s&apikey=929b4b43b3ab59b27417573fcd5868b9&hash=%s&ts=%s'%(API_LIMIT,API_OFFSET,hash.hexdigest(),str(current_timestamp)))
  response_JSON = json.loads(response_API.text)
  
  try:
      if len(response_JSON['data']['results']) != 0:
        for comic in response_JSON['data']['results']:
          characters_list = comic["characters"]["items"]
          if len(characters_list) > 0:
            for character in characters_list:
              character_id = character["resourceURI"].split("/")
              character_id = character_id[len(character_id) - 1]
              with driver.session() as session:
                try:
                  session.write_transaction(add_comic_node_and_link, comic['title'], comic["id"], character_id)
                except Exception:
                  print(Exception)
          else:
            with driver.session() as session:
              try:
                session.write_transaction(add_comic_node, comic['title'], comic["id"])
              except Exception:
                print(Exception)
        API_OFFSET += API_LIMIT
        print("********Ending iteration n. %s ***************\n" %(str(iteration_number)))
        iteration_number +=1
      else:
        results_ended = True
  except Exception:
          print(Exception)
  

KeyboardInterrupt: 

In [63]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j","1234qwer"))
while not results_ended:

  try:
      if len(response_JSON['data']['results']) != 0:
        for comic in response_JSON['data']['results']:
          characters_list = comic["characters"]["items"]
          if len(characters_list) > 0:
            for character in characters_list:
              character_id = character["resourceURI"].split("/")
              character_id = character_id[len(character_id) - 1]
              with driver.session() as session:
                try:
                  session.write_transaction(add_comic_node_and_link, comic['title'], comic["id"], character_id)
                except Exception:
                  print(Exception)
          else:
            with driver.session() as session:
              try:
                session.write_transaction(add_comic_node, comic['title'], comic["id"])
              except Exception:
                print(Exception)
        API_OFFSET += API_LIMIT
        print("********Ending iteration n. %s ***************\n" %(str(iteration_number)))
        iteration_number +=1
      else:
        results_ended = True
  except Exception:
          print(Exception)

MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)", comic_id: 1994})
MERGE (c:comic {title:"Official Handbook of the Marvel Universe (2004) #13 (TEAMS)